In [1]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Import common GIS tools
import numpy as np
import xarray as xr
# !pip install rasterio rioxarray
import rasterio.features
import rioxarray as rio

# Import Planetary Computer tools
import pystac_client
import planetary_computer as pc
import odc
from odc.stac import stac_load
from odc.algo import to_rgba
from pystac.extensions.eo import EOExtension as eo

pc.settings.set_subscription_key("st=2024-10-30T04%3A39%3A47Z&se=2024-10-31T05%3A24%3A47Z&sp=rl&sv=2024-05-04&sr=c&skoid=9c8ff44a-6a2c-4dfb-b298-1c9212f64d9a&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2024-10-31T01%3A20%3A26Z&ske=2024-11-07T01%3A20%3A26Z&sks=b&skv=2024-05-04&sig=yxW50NrgqfNEwh6gA5GpPmjbepQ0PP8d0LIG7B5GgAU%3D")


# Others
import csv
from datetime import datetime
import pandas as pd

Cloud Filtering and Masking

In [4]:
# use the bit values of the 16-bit qa_pixel flag to mask the pixels and find clouds or water
bit_flags = {
            'fill': 1<<0,
            'dilated_cloud': 1<<1,
            'cirrus': 1<<2,
            'cloud': 1<<3,
            'shadow': 1<<4,
            'snow': 1<<5,
            'clear': 1<<6,
            'water': 1<<7
}

# func: mask pixels with a give type:
def get_flags_to_mask(mask, flags):
   # Initialize a mask to store results with the same shape as the input mask
  combine_flag_mask = np.zeros_like(mask, dtype=bool)
  for flag in flags:
    # Compute the mask for the current flag
    current_flag_mask = np.bitwise_and(mask, bit_flags[flag])>0
    # Combine the current flag mask with the overall result mask
    combine_flag_mask = combine_flag_mask | current_flag_mask
  return combine_flag_mask

In [5]:
# find window_size
def find_window_size(season, date, file_csv, lat_str):
    is_found_start_date = False
    if season == "SA":
        # Search for the harvest start date (only for SA season)
        with open(file_csv, 'r') as file_csv:
            csv_reader = csv.DictReader(file_csv)
            for row in csv_reader:
              if row["Latitude"] == lat_str and row['Season(SA = Summer Autumn, WS = Winter Spring)'] == "WS":
                  start_date = row['Date of Harvest']
                  is_found_start_date = True
                  break

    date_obj = datetime.strptime(date, '%d-%m-%Y')
    doh = date_obj.strftime('%Y-%m-%d')  # date of harvest
    if season == "SA":
        if is_found_start_date:
            start_date_obj = pd.to_datetime(start_date, dayfirst=True)
            new_start_date = start_date_obj.strftime('%Y-%m-%d')
            window_size = f"{max(new_start_date, '2022-04-01')}/{doh}"
        else:
            window_size = f"2022-04-01/{doh}"
    elif season == "WS":
        window_size = f"2021-11-01/{doh}"
    return window_size

In [6]:
def process_data(file_csv, longitude, latitude, season, date, box_deg=0.10):
  catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

  lat_str = str(latitude)
  window_size = find_window_size(season, date, file_csv, lat_str)

  bbox = [
      longitude-box_deg/2,
      latitude-box_deg/2,
      longitude+box_deg/2,
      latitude+box_deg/2]

  # Search for relevant satellite images from the catalog
  search = catalog.search(
      collections=["landsat-c2-l2"],
      bbox=bbox,
      datetime=window_size,
      query={
          'eo:cloud_cover': {"lt": 10}, # 10% clould cover
          'platform': {"in": ["landsat-8", "landsat-9"]},})
  items = search.get_all_items()
  if not items:
        print("No images found.")
        return np.nan, np.nan, np.nan, np.nan

  print(f"Found {len(items)} images on {latitude}")


    # Select the image with the least cloud cover
  selected_item = min(items, key=lambda item: eo.ext(item).cloud_cover)
  bands_interest = ['red', 'nir08', 'qa_pixel', 'green', 'blue', 'swir16']

  xx = stac_load(
        [selected_item],
        bands = bands_interest,
        crs='EPSG:4326',
        resolution=30/111320,
        patch_url=pc.sign,
        bbox=bbox,).isel(time=0)

    # Apply scaling and offset for the bands (https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2)
  xx['red'] = xx['red']*0.0000275 - 0.2
  xx['nir08'] = xx['nir08']*0.0000275 - 0.2
  xx['green'] = xx['green']*0.0000275 - 0.2
  xx['blue'] = xx['blue']*0.0000275 - 0.2
  xx['swir16'] = xx['swir16']*0.0000275 - 0.2

    # Mask invalid data
  quality_mask = get_flags_to_mask(xx['qa_pixel'], ['fill', 'dilated_cloud', 'cirrus', 'cloud', 'shadow', 'water'])
  masked_data = xx.where(~quality_mask)
  clean_data = masked_data.mean(dim=['longitude', 'latitude']).compute()

    # Cleaned bands
  nir08 = clean_data.nir08.item()
  red = clean_data.red.item()
  green = clean_data.green.item()
  blue = clean_data.blue.item()
  swir16 = clean_data.swir16.item()

    # calculate index
  ndvi = (nir08 - red) / (nir08 + red)
  ndwi = (green - swir16) / (green + swir16)
  avi = np.power((nir08*(1-red)*(nir08 - red)), 1/3)
  ndmi = (nir08 - swir16) / (nir08 + swir16)

  return ndvi, ndwi, avi, ndmi

In [8]:
ndvi, ndwi, avi, ndmi = process_data('../data/Crop_Yield_Data.csv',105.248554,10.510542, "SA", "15-07-2022"  )
print(ndvi, ndwi, avi, ndmi)

Found 1 images on 10.510542
0.6660664683945674 -0.34380810801863937 0.45145094534199015 0.33355161059081007
